In [ ]:
!pip install nvcc4jupyter

In [ ]:
%load_ext nvcc4jupyter

A programe for vector addition. Reference: https://github.com/PacktPublishing/Learn-CUDA-Programming

In [ ]:
%%cuda
#include <stdio.h>
#include <stdlib.h>
#include <type_traits>

#define TYPE int
#define N 512

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
__global__ void device_add(T *a, T *b, T *c)
{

    int id = threadIdx.x + blockIdx.x * blockDim.x;
    c[id] = a[id] + b[id];
}

template <typename T, typename = std::enable_if_t<std::is_arithmetic<T>::value>>
void print_output(T *a, T *b, T *c)
{
    for (int i = 0; i < N; ++i)
    {
        printf("\n %d + %d  = %d", a[i], b[i], c[i]);
    }
}

int main()
{
    TYPE *a, *b, *c;
    TYPE *d_a, *d_b, *d_c;
    int threads_per_block = 0, no_of_blocks = 0;

    int size = N * sizeof(TYPE);

    // Allocate space for host copies of a, b, c and setup input values
    a = (TYPE *)malloc(size);
    b = (TYPE *)malloc(size);
    c = (TYPE *)malloc(size);

    for (int i = 0; i < N; ++i)
    {
        a[i] = i;
        b[i] = i;
    }

    // Allocate space for device copies of a, b, c
    cudaMalloc((void **)&d_a, size);
    cudaMalloc((void **)&d_b, size);
    cudaMalloc((void **)&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    threads_per_block = 4;
    no_of_blocks = (N + threads_per_block - 1) / threads_per_block;
    device_add<<<no_of_blocks, threads_per_block>>>(d_a, d_b, d_c);

    // Copy result back to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    print_output(a, b, c);

    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}
